In [103]:
from copy import copy
from aocd.models import Puzzle
from collections import Counter
puzzle = Puzzle(year=2021, day=19)
data = puzzle.input_data.split('\n')

In [100]:
data = '''
--- scanner 0 ---
404,-588,-901
528,-643,409
-838,591,734
390,-675,-793
-537,-823,-458
-485,-357,347
-345,-311,381
-661,-816,-575
-876,649,763
-618,-824,-621
553,345,-567
474,580,667
-447,-329,318
-584,868,-557
544,-627,-890
564,392,-477
455,729,728
-892,524,684
-689,845,-530
423,-701,434
7,-33,-71
630,319,-379
443,580,662
-789,900,-551
459,-707,401

--- scanner 1 ---
686,422,578
605,423,415
515,917,-361
-336,658,858
95,138,22
-476,619,847
-340,-569,-846
567,-361,727
-460,603,-452
669,-402,600
729,430,532
-500,-761,534
-322,571,750
-466,-666,-811
-429,-592,574
-355,545,-477
703,-491,-529
-328,-685,520
413,935,-424
-391,539,-444
586,-435,557
-364,-763,-893
807,-499,-711
755,-354,-619
553,889,-390

--- scanner 2 ---
649,640,665
682,-795,504
-784,533,-524
-644,584,-595
-588,-843,648
-30,6,44
-674,560,763
500,723,-460
609,671,-379
-555,-800,653
-675,-892,-343
697,-426,-610
578,704,681
493,664,-388
-671,-858,530
-667,343,800
571,-461,-707
-138,-166,112
-889,563,-600
646,-828,498
640,759,510
-630,509,768
-681,-892,-333
673,-379,-804
-742,-814,-386
577,-820,562

--- scanner 3 ---
-589,542,597
605,-692,669
-500,565,-823
-660,373,557
-458,-679,-417
-488,449,543
-626,468,-788
338,-750,-386
528,-832,-391
562,-778,733
-938,-730,414
543,643,-506
-524,371,-870
407,773,750
-104,29,83
378,-903,-323
-778,-728,485
426,699,580
-438,-605,-362
-469,-447,-387
509,732,623
647,635,-688
-868,-804,481
614,-800,639
595,780,-596

--- scanner 4 ---
727,592,562
-293,-554,779
441,611,-461
-714,465,-776
-743,427,-804
-660,-479,-426
832,-632,460
927,-485,-438
408,393,-506
466,436,-512
110,16,151
-258,-428,682
-393,719,612
-211,-452,876
808,-476,-593
-575,615,604
-485,667,467
-680,325,-822
-627,-443,-432
872,-547,-609
833,512,582
807,604,487
839,-516,451
891,-625,532
-652,-548,-490
30,-46,-14
'''
data = data.split('\n')[1:-1]

In [104]:
scanners = []
for line in data:
    if line == '':
        continue
    elif line[:3] == '---':
        scanners.append([])
    else:
        scanners[-1].append([int(c) for c in line.split(',')])

orientations = set()
c = (1, 2, 3)
for _ in range(4):
    c = (c[1], - c[0], c[2])
    for _ in range(4):
        c = (c[2], c[1], - c[0])
        for _ in range(4):
            c = (c[0], - c[2], c[1])
            orientations.add(c)

def sign(x):
    if x < 0:
        return - 1
    elif x > 0:
        return 1
    else:
        return 0

def difference(a, b):
    return [i - j for i, j in zip(a, b)]

class Transformation:
    def __init__(self, p=(1, 2, 3), v=(0, 0, 0)):
        self.p = p
        self.v = v

    def __repr__(self):
        return f'p{self.p}+{self.v}'

    def __call__(self, x):
        return [sign(p) * x[abs(p) - 1] + v for p, v in zip(self.p, self.v)]

    def inverse(self):
        p = [None, None, None]
        for j, i in enumerate(self.p):
            p[abs(i) - 1] = sign(i) * (j + 1)
        v = [- sign(pi) * self.v[abs(pi) - 1] for pi in p]
        return Transformation(p, v)


def match(beacons_a, beacons_b):
    for orientation in orientations:
        t = Transformation(orientation)
        vectors = []
        for a in beacons_a:
            for b in beacons_b:
                v = difference(a, t(b))
                vectors.append(tuple(v))
        counts = Counter(vectors)
        counts.most_common()
        translation, n = counts.most_common(1)[0]
        if n >= 12:
            return Transformation(orientation, translation)
    return

transformations = {}
for i in range(len(scanners)):
    for j in range(i + 1, len(scanners)):
        transformation = match(scanners[i], scanners[j])
        if transformation:
            transformations[(j, i)] = transformation
            transformations[(i, j)] = transformation.inverse()

def find_path(path):
    if path[-1] == 0:
        return path
    for (a, b), t in transformations.items():
        if path[-1] == a and b not in path:
            new_path = find_path(path + [b])
            if new_path:
                return new_path

all_beacons = []
for i, beacons in enumerate(scanners):
    path = find_path([i])
    for a, b in zip(path[:-1], path[1:]):
        beacons = [transformations[(a, b)](beacon) for beacon in beacons]
    all_beacons.extend(beacons)

all_beacons = set([tuple(b) for b in all_beacons])
print(f'There are {len(all_beacons)} beacons.')

There are 457 beacons.


In [105]:
puzzle.answer_a = 457

In [117]:
from itertools import permutations

scanner_positions = []
for i in range(len(scanners)):
    path = find_path([i])
    base = [0, 0, 0]
    for a, b in zip(path[:-1], path[1:]):
        base = transformations[(a, b)](base)
    scanner_positions.append(base)

max_distance = 0
for a, b in permutations(scanner_positions, 2):
    distance = sum([abs(i - j) for i, j in zip(a, b)])
    max_distance = max(max_distance, distance)

print(f'Maximum Manhattan distance: {max_distance}.')

Maximum Manhattan distance: 13243.


In [115]:
puzzle.answer_b = 13243

That's the right answer!  You are one gold star closer to finding the sleigh keys.You have completed Day 19! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
Opening in existing browser session.


[6650:6650:0100/000000.374103:ERROR:sandbox_linux.cc(376)] InitializeSandbox() called with multiple threads in process gpu-process.
